In [22]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

# Groupby apply on large (relational) data set

In [23]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


# Test Data

In [24]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]
test_data.head()

,date,state,country,confirmed
15737,2020-03-21,no,Germany,22213.0
15777,2020-03-21,no,US,24508.0
16003,2020-03-22,no,Germany,24873.0
16042,2020-03-22,no,US,33152.0
16269,2020-03-23,no,Germany,29056.0


In [25]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-09-04,no,250283.0
US,2020-09-04,no,6200518.0


In [26]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [27]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-09-04   250283.0
      US      2020-09-04  6200518.0

In [28]:
def rolling_reg(df_input,col='confirmed'):
    
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [29]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15737            NaN
                16003            NaN
                16269       7.417994
                16534       7.142035
                16801       8.012983
                17067       6.954407
                17331       6.501919
                17598       7.390371
                17864      10.136671
                18129      13.541893
                18397      13.781393
                18662      13.140681
                18929      12.037271
                19193      12.735506
                19460      16.052694
                19726      21.372453
                19991      27.427355
                20259      27.511936
                20524      21.792179
                20790      21.495848
                21057      26.565108
                21323      36.198404
                21589      43.982990
                21855      49.423444
                22121      74.043747
                22387      56.424411
                

In [30]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [31]:

pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1263,NaN


In [32]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [33]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


# Filtering the data with groupby apply

In [34]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [35]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()


In [36]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


# Filtering doubling rate

In [37]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
60377,no,Zimbabwe,59146,124.782721
60378,no,Zimbabwe,59382,124.595147
60379,no,Zimbabwe,59678,94.218077
60380,no,Zimbabwe,59914,80.534870
60381,no,Zimbabwe,60381,84.126408


In [38]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
60377,60377,2020-09-04,no,Barbados,178.0,177.000000,178.0,196.777778
60378,60378,2020-09-04,no,Belarus,72485.0,420.403101,72471.4,445.252463
60379,60379,2020-09-04,no,Belgium,87174.0,137.043547,87057.0,175.478816
60380,60380,2020-09-04,no,Albania,9967.0,82.396095,9960.8,85.917976
60381,60381,2020-09-04,no,Zimbabwe,6837.0,67.514238,6801.6,84.126408


In [39]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)

In [40]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
59093,59093,2020-08-31,no,Germany,244802.0,247.734960,244873.6,227.240190
59361,59361,2020-09-01,no,Germany,246015.0,180.595818,246074.6,216.936758
59625,59625,2020-09-02,no,Germany,247411.0,188.636259,247470.2,189.585971
59893,59893,2020-09-03,no,Germany,248840.0,175.166018,248848.9,178.397842
60158,60158,2020-09-04,no,Germany,250283.0,173.290158,250227.6,180.495322


In [41]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)